In [1]:
import pandas as pd
import numpy as np
import simpletransformers

In [2]:
data = pd.read_csv("/home/shriya/Documents/NER/SimpleTransformer/hebbsNLP/hebbsNLP/dataset.csv")
data.head()

,Sentence#,Main-Line#,Sub-Line#,Word,POS,Tag
0,1,1,0,',RD_PUNC,O
1,1,2,0,ಆವಿಷ್ಕಾರ,N_NN,O
2,1,3,0,",",RD_PUNC,O
3,1,4,0,ಸುಸ್ಥಿರತೆ,N_NN,O
4,1,5,0,ಮತ್ತು,CC_CCD,O


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    

In [4]:
data["Sentence#"] = LabelEncoder().fit_transform(data["Sentence#"] )
data.head()

,Sentence#,Main-Line#,Sub-Line#,Word,POS,Tag
0,0,1,0,',RD_PUNC,O
1,0,2,0,ಆವಿಷ್ಕಾರ,N_NN,O
2,0,3,0,",",RD_PUNC,O
3,0,4,0,ಸುಸ್ಥಿರತೆ,N_NN,O
4,0,5,0,ಮತ್ತು,CC_CCD,O


In [5]:
data.rename(columns={"Sentence#":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
data["labels"] = data["labels"].str.upper()
data["words"] = data["words"].astype(str)
data.isnull().sum()

sentence_id    0
Main-Line#     0
Sub-Line#      0
words          0
POS            0
labels         0
dtype: int64

In [6]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [8]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [9]:
test_data
train_data

,sentence_id,words,labels
470,27,ಒಂದು,O
32,2,",",O
529,30,ಮತ್ತು,O
654,35,ಪರಿಹಾರಗಳನ್ನು,O
892,44,ಬೆಂಬಲಿಸಲು,O
...,...,...,...
907,45,ಮಹೀಂದ್ರಾದಲ್ಲಿ,NEO
512,29,ಮತ್ತು,O
969,48,ನಿಕಟ,O
75,6,ವಿಶೇಷ,O


In [10]:
from simpletransformers.ner import NERModel,NERArgs
     

2023-06-24 12:29:04.655416: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-24 12:29:04.686263: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-24 12:29:04.836273: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-24 12:29:04.837917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 12:29:05.533581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [11]:
label = data["labels"].unique().tolist()
label

['O', 'NEL', 'NETE', 'NED', 'NEN', 'NEP', 'NEO', 'NEA']

In [12]:
#training args(hyperparameters)
args = NERArgs()
args.num_train_epochs = 6
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.weight_decay = 0.01
args.dropout = 0.01

In [13]:
model = NERModel('bert' , 'bert-base-multilingual-cased', args=args, labels=label, use_cuda=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [14]:
model.train_model(train_data,eval_data=test_data , acc=accuracy_score)

/home/shriya/anaconda3/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 2 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 3 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 4 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 5 of 6:   0%|          | 0/2 [00:00<?, ?it/s]

(12, 0.9110313430428505)

In [15]:
result, model_outputs, preds_list = model.eval_model(test_data)

/home/shriya/anaconda3/lib/python3.10/site-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

/home/shriya/anaconda3/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NETE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shriya/anaconda3/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NEL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shriya/anaconda3/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NEP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shriya/anaconda3/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NEO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shriya/anaconda3/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NEN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/shriya/anaconda3/l

In [16]:
result

{'eval_loss': 0.5396615862846375,
 'precision': 0.8888888888888888,
 'recall': 0.1951219512195122,
 'f1_score': 0.32}